# Music tagging Transformer
This file traines a transformer from the melspectogram features


In [2]:
import json
import yaml
from pathlib import Path
import save_utils
from plot import plot_hist, plot_conf_mat

import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import (
    Input,
    GlobalAvgPool1D,
    Dense,
    Dropout,
)
from keras.models import Model, load_model
from keras.optimizers import Adam

from transformer import Encoder

from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import clip_ops

## Define crossentropy and accuracy metric for the training routine
For the metric we use a binary accuracy, for the loss a binary cross-entropy loss

In [3]:
def custom_binary_accuracy(y_true, y_pred, threshold=0.5):
    """Calculating the accuracy for the model where the treshold is 0.5 and the result will be between 0 and 1
    """
    threshold = math_ops.cast(threshold, y_pred.dtype)
    y_pred = math_ops.cast(y_pred > threshold, y_pred.dtype)
    y_true = math_ops.cast(y_true > threshold, y_true.dtype)

    return K.mean(math_ops.equal(y_true, y_pred), axis=-1)


def custom_binary_crossentropy(y_true, y_pred):
    """Calculating the cross entropy from probabilities where the result will be between 0 and 1
    """
    y_pred = ops.convert_to_tensor(y_pred)
    y_true = math_ops.cast(y_true, y_pred.dtype)
    epsilon_ = K._constant_to_tensor(K.epsilon(), y_pred.dtype.base_dtype)
    output = clip_ops.clip_by_value(y_pred, epsilon_, 1.0 - epsilon_)

    # Compute cross entropy from probabilities.
    bce = 4 * y_true * math_ops.log(output + K.epsilon())
    bce += (1 - y_true) * math_ops.log(1 - output + K.epsilon())
    return K.sum(-bce, axis=-1)

## Define model
Define the transformer model structure with the Encoder from the transformer util file.

In [4]:
def transformer_model(model_config, n_classes):
    """Transformer model where the model will be trained on the training data
    """
    num_layers = model_config['n_layers']
    d_model = model_config['d_model']
    num_heads = model_config['n_heads']
    dff = model_config['dff']
    maximum_position_encoding = model_config['max_pos_encoding']
    init_lr = model_config['init_learning_rate']
    dropout_rate = model_config['dropout_rate']
    activations = model_config['activations']

    input_layer = Input((None, d_model))
    #Getting the encoder code from the transformer
    encoder = Encoder(
        num_layers=num_layers,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff,
        maximum_position_encoding=maximum_position_encoding,
        rate=model_config['encoder_rate']
    )

    x = encoder(input_layer)
    x = Dropout(dropout_rate)(x)
    x = GlobalAvgPool1D()(x)
    x = Dense(4 * n_classes, activation=activations[0])(x)

    out = Dense(n_classes, activation=activations[1])(x)

    model = Model(inputs=input_layer, outputs=out)
    model.compile(optimizer=Adam(init_lr), loss=custom_binary_crossentropy, metrics=[custom_binary_accuracy])
    model.summary()
    return model


# Main routine
## Load the configuration for this model

In [5]:
# read in the config
with open('music_tag_transformer/transformer_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Extract the values
transformer_name = config['transformer_name']
transformer_pretrained_name = config['pretrained_transformer']
batch_size = config['batch_size']
epochs = config['epochs']
data_dir = Path(config['data_dir'])

## Load preprocessed data
Load the numpy arrays and the label-class mapping. Split the data accordingly into train, validation and test set

In [6]:
melspec_data = save_utils.load_sliced_numpy_array('melspec_features', data_dir=data_dir)
labels = np.load(data_dir/'labels.npy')

with open(data_dir/'class_label_index_mapping.json', 'r') as f:
    labels_to_id = json.load(f)

mel_train, mel_test_val, lab_train, lab_test_val = train_test_split(melspec_data, labels, train_size=config['train_set_size'], random_state=config['random_state'])
mel_val, mel_test, lab_val, lab_test             = train_test_split(mel_test_val, lab_test_val, test_size=(config['val_set_size']/(1-config['train_set_size'])), shuffle=False)

# Check the shapes of the splitted sets
assert mel_train.shape[0] == lab_train.shape[0] and mel_test.shape[0] == lab_test.shape[0] and mel_val.shape[0] == lab_val.shape[0]
assert mel_train.shape[1] == mel_test.shape[1] == mel_val.shape[1] and lab_train.shape[1] == lab_test.shape[1] == lab_val.shape[1]

Loaded 12 files:
    melspec_features_001.npy
    melspec_features_002.npy
    melspec_features_003.npy
    melspec_features_007.npy
    melspec_features_012.npy
    melspec_features_006.npy
    melspec_features_010.npy
    melspec_features_004.npy
    melspec_features_005.npy
    melspec_features_011.npy
    melspec_features_008.npy
    melspec_features_009.npy


## Set up the model callbacks
For the initializing parameters, refer to the config file.  
For callbacks, we use a frequently backup of the model as checkpoints, and early stopping mechanism to prevent overfitting on the training data and a learningrate reducer, that smallers the update steps when the validation metric does not improve any more

In [7]:
train_config = config['training']

checkpoint = ModelCheckpoint(
    transformer_name,
    monitor=train_config['monitor'],
    verbose=1,
    save_best_only=train_config['save_best_weights'],
    mode=train_config['monitor_mode'],
    save_weights_only=False
)

early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    verbose = 1,
    mode = 'min',
    restore_best_weights = True,
    start_from_epoch = 5
)

# Reduce learning rate when val_loss stopps improving
lr_reduce_config = train_config['lr_reducing']
lr_reducing_on_platteau = ReduceLROnPlateau(
    monitor=lr_reduce_config['monitor'], patience=lr_reduce_config['patience'], min_lr=lr_reduce_config['min_lr'], mode=lr_reduce_config['mode']
)

## Train routine

In [8]:
# read in the config
with open('preprocess_config.yaml', 'r') as f:
    pp_config = yaml.safe_load(f)

n_mels = pp_config['melspectogram']['n_mels']
mel_train = mel_train.reshape(mel_train.shape[0], -1, n_mels)
mel_val   = mel_val.reshape(mel_val.shape[0], -1, n_mels)

model = transformer_model(config['model_structure'], n_classes=len(labels_to_id))

history = model.fit(
        x=mel_train,
        y=lab_train,
        validation_data=(mel_val, lab_val),
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[checkpoint, lr_reducing_on_platteau, early_stopping],
        use_multiprocessing=True,
        verbose=2
    )

model.save('models/complete'+transformer_name)

2023-01-12 17:50:54.717651: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 128)]       0         
                                                                 
 encoder (Encoder)           (None, None, 128)         529920    
                                                                 
 dropout_9 (Dropout)         (None, None, 128)         0         
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_24 (Dense)            (None, 20)                2580      
                                                                 
 dense_25 (Dense)            (None, 5)                 105       
                                                             

dump the history to file

In [ ]:
with open(f'models/{transformer_name}_history{epochs}', 'wb') as f:
    pickle.dump(history.history, f)

Evaluate the training history

In [ ]:
with open(f'models/{transformer_name}_history{epochs}', "rb") as f:
    history_dict = pickle.load(f)

print('available keys', history_dict.keys())

#Plots for the accuracies and losses of the train and validation data per epoch
plot_hist(history_dict, ('accuracy', 'val_accuracy'), legends=('train', 'validation'), title='Accuracy', y_label='accuracy ->', x_label='epochs ->', save_to=f'Plots/short_chunk_cnn_{epochs}_acuracy')
plot_hist(history_dict, ('loss', 'val_loss'), legends=('train', 'validation'), title='Loss', y_label='loss ->', x_label='epochs ->', save_to=f'Plots/short_chunk_cnn_{epochs}_loss')

Predict model output on all the available sets

In [ ]:
predict = lambda model, data: np.argmax(model.predict(data), axis=-1)

#Loading in the model
model = load_model(f"models/{transformer_name}{epochs}.h5")

# Training prediction
y_pred_train = predict(model, mel_train)
y_true_train = np.argmax(lab_train, axis= -1)
print(f"ACCURACY FOR TRAIN SET {accuracy_score(y_true_train, y_pred_train)*100:.4f} %")
print(f"MACRO F1 SCORE FOR TRAIN SET {f1_score(y_true_train, y_pred_train, average='macro')*100:.4f} %")
print(f"MICRO F1 SCORE FOR TRAIN SET {f1_score(y_true_train, y_pred_train, average='micro')*100:.4f} %")


# Validation prediction
y_pred_val = predict(model, mel_val)
y_true_val = np.argmax(lab_val, axis= -1)
print(f"ACCURACY FOR VAL SET {accuracy_score(y_true_val, y_pred_val)*100:.4f} %")
print(f"MACRO F1 SCORE FOR VAL SET {f1_score(y_true_val, y_pred_val, average='macro')*100:.4f} %")
print(f"MICRO F1 SCORE FOR VAL SET {f1_score(y_true_val, y_pred_val, average='micro')*100:.4f} %")

# Test prediction
y_pred_test = predict(model, mel_test)
y_true_test = np.argmax(lab_test, axis= -1)
print(f"ACCURACY FOR TEST SET {accuracy_score(y_true_test, y_pred_test)*100:.4f} %")
print(f"MACRO F1 SCORE FOR TEST SET {f1_score(y_true_test, y_pred_test, average='macro')*100:.4f} %")
print(f"MICRO F1 SCORE FOR TEST SET {f1_score(y_true_test, y_pred_test, average='micro')*100:.4f} %")

# Plot confusion matrix

In [ ]:
with open('data/class_label_index_mapping.json', 'r') as f:
    class_names = json.load(f).keys()

#Confusion matrix of the predicted labels versus the true labels
conf_mat = confusion_matrix(y_true_test, y_pred_test, normalize= 'true')
conf_mat = np.round(conf_mat, 2)

conf_mat_df = pd.DataFrame(conf_mat, columns=class_names, index=class_names)

plot_conf_mat(conf_mat_df, save_to=f"Plots/{transformer_name}{epochs}_test_conf_mat.png")